# **Encoding**

In [8]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import pickle


import sys
import warnings
warnings.filterwarnings('ignore')

sys.path.append('../')
from src import funciones_encoding as fe

pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
df = pd.read_pickle("../datos/dataframes/df_nonulls.pkl")
df.reset_index(drop=True, inplace=True)
df.head(2)

,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,JobInvolvement,PerformanceRating
0,3.0,4.0,2.0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,Female,1,Healthcare Representative,Married,131160,1.0,11,0,1.0,6,1,0,0,3,3
1,3.0,2.0,4.0,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,Female,1,Research Scientist,Single,41890,0.0,23,1,6.0,3,5,1,4,2,4


In [10]:
df.select_dtypes(exclude=np.number).columns

Index(['EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance',
       'Attrition', 'BusinessTravel', 'Department', 'Education',
       'EducationField', 'Gender', 'JobLevel', 'JobRole', 'MaritalStatus',
       'StockOptionLevel', 'TrainingTimesLastYear', 'JobInvolvement',
       'PerformanceRating'],
      dtype='object')

Cuando tengamos un problema de clasificación para hacer el encoding y ver si entre las variables categóricas hay diferencias significativas usaremos tablas de contingencia.

In [11]:
lista_col_categoricas = df.select_dtypes(exclude=np.number).columns

cols_con_orden = []
cols_sin_orden = []

for col in lista_col_categoricas:
    test = fe.TestEstadisticos(df, "Attrition", col)
    if test.detectar_orden_problema_categorico():
        cols_con_orden.append(col)
    else:
        cols_sin_orden.append(col)

print(f"Las columnas con orden son {cols_con_orden}")
print(f"Las columnas que NO tienen orden son {cols_sin_orden}")

Estamos comparando las variables de ENVIRONMENTSATISFACTION
El p-valor es 0.0002460296911026889
La variable ENVIRONMENTSATISFACTION SÍ tiene orden ✅

---------------------------------------------

Estamos comparando las variables de JOBSATISFACTION
El p-valor es 0.0004135653008694639
La variable JOBSATISFACTION SÍ tiene orden ✅

---------------------------------------------

Estamos comparando las variables de WORKLIFEBALANCE
El p-valor es 0.0006966478970883569
La variable WORKLIFEBALANCE SÍ tiene orden ✅

---------------------------------------------

Estamos comparando las variables de BUSINESSTRAVEL
El p-valor es 4.676256122046688e-07
La variable BUSINESSTRAVEL SÍ tiene orden ✅

---------------------------------------------

Estamos comparando las variables de DEPARTMENT
El p-valor es 0.0027746683432808813
La variable DEPARTMENT SÍ tiene orden ✅

---------------------------------------------

Estamos comparando las variables de EDUCATION
El p-valor es 0.5874678447175496
La variable 

A las columnas con orden les vampos a aplicar el Target Encoder, las columnas sin orden las vamos a devolver a su estado numérico a todas las que sean posibles y a hacer onehot encoder a Gender. Si las méstricas obtenidas no fueran muy buenas podríamos volver a este paso y hacer one hot encoder a todas las que no tuvieran orden.

In [12]:
cols_cat_a_num = ['Education', 'JobLevel', 'StockOptionLevel', 'PerformanceRating', "TrainingTimesLastYear", "JobInvolvement"]
df[cols_cat_a_num] = df[cols_cat_a_num].astype("int")

df["Attrition"] = df["Attrition"].map({"Yes":1, "No":0})

df.head(2)

,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,JobInvolvement,PerformanceRating
0,3.0,4.0,2.0,51,0,Travel_Rarely,Sales,6,2,Life Sciences,Female,1,Healthcare Representative,Married,131160,1.0,11,0,1.0,6,1,0,0,3,3
1,3.0,2.0,4.0,31,1,Travel_Frequently,Research & Development,10,1,Life Sciences,Female,1,Research Scientist,Single,41890,0.0,23,1,6.0,3,5,1,4,2,4


In [13]:
diccionario_encoding={"onehot":["Gender", 'JobRole'], "frequency":['EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance', 'BusinessTravel', 'Department', 'EducationField',  'MaritalStatus']}

var_respuesta = "Attrition"
encoding = fe.Encoding(df, diccionario_encoding, var_respuesta)

# -------------- One hot encoder --------------
df_ohe, one_hot_encoder = encoding.one_hot_encoding()
df_ohe.drop(columns=diccionario_encoding["onehot"], inplace=True)

with open('../datos/preprocesamiento/one_hot_encoder.pkl', 'wb') as f:
    pickle.dump(one_hot_encoder, f)

df_ohe.head()

# -------------- Target encoder --------------
df_ohe_freq = encoding.frequency_encoding()

# with open('../datos/preprocesamiento/frequency_encoder.pkl', 'wb') as f:
#     pickle.dump(frequency_encoder, f)

df_ohe_freq.head(3)

,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,JobLevel,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,JobInvolvement,PerformanceRating,Gender_Female,Gender_Male,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative
0,0.304854,0.311974,0.228479,51,0,0.710032,0.303560,6,2,0.416181,1,0.458252,131160,1.0,11,0,1.0,6,1,0,0,3,3,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.304854,0.187055,0.100324,31,1,0.188350,0.655016,10,1,0.416181,1,0.317799,41890,0.0,23,1,6.0,3,5,1,4,2,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.192880,0.187055,0.053074,32,0,0.188350,0.655016,17,4,0.056958,4,0.458252,193280,1.0,15,3,5.0,2,5,0,3,3,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Guardamos el df la con el encoding hecho:

In [14]:
df_ohe_freq.to_pickle("../datos/dataframes/df_nonulls_encoded.pkl")